## Lab | Cross Validation
For this lab, we will build a model on customer churn binary classification problem. You will be using files_for_lab/Customer-Churn.csv file.

Instructions
Apply SMOTE for upsampling the data

- Use logistic regression to fit the model and compute the accuracy of the model.   
- Use decision tree classifier to fit the model and compute the accuracy of the model.   
- Compare the accuracies of the two models.   
- Apply TomekLinks for downsampling   

- It is important to remember that it does not make the two classes equal but only removes the points from the majority class that are close to other points in minority class.   
- Use logistic regression to fit the model and compute the accuracy of the model.   
- Use decision tree classifier to fit the model and compute the accuracy of the model.   
- Compare the accuracies of the two models.   
- You can also apply this algorithm one more time and check the how the imbalance in the two classes changed from the last time.   

In [31]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from imblearn.under_sampling import TomekLinks
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [2]:
data=pd.read_csv('Customer-churn.csv')
data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [5]:
#aplicamso cambios anterior lab
data['TotalCharges']=pd.to_numeric(data['TotalCharges'], errors='coerce')
data['TotalCharges'] =data['TotalCharges'].fillna(data['TotalCharges'].mean())

In [7]:
#dividimos data

num = data.select_dtypes("number")
cat = data.select_dtypes("object") # Vamos a aplicarle el label encoder tambien a la y para poder trabajar con ella en varios modelos
cat.head()

,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,Churn
0,Female,Yes,No,No,No,Yes,No,No,No,No,Month-to-month,No
1,Male,No,No,Yes,Yes,No,Yes,No,No,No,One year,No
2,Male,No,No,Yes,Yes,Yes,No,No,No,No,Month-to-month,Yes
3,Male,No,No,No,Yes,No,Yes,Yes,No,No,One year,No
4,Female,No,No,Yes,No,No,No,No,No,No,Month-to-month,Yes


In [9]:
# Trabajamos con la categorica primero
cat.isnull().sum()

gender              0
Partner             0
Dependents          0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
Churn               0
dtype: int64

In [10]:
# Aplicamos Label Encoder
cat=cat.apply(preprocessing.LabelEncoder().fit_transform)
cat

,gender,Partner,Dependents,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,Churn
0,0,1,0,0,0,2,0,0,0,0,0,0
1,1,0,0,1,2,0,2,0,0,0,1,0
2,1,0,0,1,2,2,0,0,0,0,0,1
3,1,0,0,0,2,0,2,2,0,0,1,0
4,0,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,1,1,1,2,0,2,2,2,2,1,0
7039,0,1,1,1,0,2,2,0,2,2,1,0
7040,0,1,1,0,2,0,0,0,0,0,0,0
7041,1,1,0,1,0,0,0,0,0,0,0,1


In [11]:
num

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65
...,...,...,...,...
7038,0,24,84.80,1990.50
7039,0,72,103.20,7362.90
7040,0,11,29.60,346.45
7041,1,4,74.40,306.60


In [12]:
# Vamos a aplicar el Standar Scaler a nuestra data numerica
Standard=StandardScaler()
Standard=Standard.fit_transform(num)
Standard=pd.DataFrame(Standard) 
Standard

,0,1,2,3
0,-0.439916,-1.277445,-1.160323,-0.994971
1,-0.439916,0.066327,-0.259629,-0.173876
2,-0.439916,-1.236724,-0.362660,-0.960399
3,-0.439916,0.514251,-0.746535,-0.195400
4,-0.439916,-1.236724,0.197365,-0.941193
...,...,...,...,...
7038,-0.439916,-0.340876,0.665992,-0.129281
7039,-0.439916,1.613701,1.277533,2.242808
7040,-0.439916,-0.870241,-1.168632,-0.855182
7041,2.273159,-1.155283,0.320338,-0.872777


In [13]:
#Concatenamos data
data=np.concatenate((Standard,cat), axis=1)
data=pd.DataFrame(data)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.439916,-1.277445,-1.160323,-0.994971,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.439916,0.066327,-0.259629,-0.173876,1.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0
2,-0.439916,-1.236724,-0.362660,-0.960399,1.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.439916,0.514251,-0.746535,-0.195400,1.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0
4,-0.439916,-1.236724,0.197365,-0.941193,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,-0.439916,-0.340876,0.665992,-0.129281,1.0,1.0,1.0,1.0,2.0,0.0,2.0,2.0,2.0,2.0,1.0,0.0
7039,-0.439916,1.613701,1.277533,2.242808,0.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0,1.0,0.0
7040,-0.439916,-0.870241,-1.168632,-0.855182,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7041,2.273159,-1.155283,0.320338,-0.872777,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
# Renombramos nuestras columnas 
data=data.set_axis(['seniorcitizen','tenure','monthlycharges','totalcharges','gender','partner','dependents','phoneservice','onlinesecurity','onlinebackup','deviceprotection','techsupport','streamingtv','streamingmovies','contract','churn'], axis=1)
data

,seniorcitizen,tenure,monthlycharges,totalcharges,gender,partner,dependents,phoneservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,churn
0,-0.439916,-1.277445,-1.160323,-0.994971,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.439916,0.066327,-0.259629,-0.173876,1.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0
2,-0.439916,-1.236724,-0.362660,-0.960399,1.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.439916,0.514251,-0.746535,-0.195400,1.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,0.0,1.0,0.0
4,-0.439916,-1.236724,0.197365,-0.941193,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,-0.439916,-0.340876,0.665992,-0.129281,1.0,1.0,1.0,1.0,2.0,0.0,2.0,2.0,2.0,2.0,1.0,0.0
7039,-0.439916,1.613701,1.277533,2.242808,0.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0,1.0,0.0
7040,-0.439916,-0.870241,-1.168632,-0.855182,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7041,2.273159,-1.155283,0.320338,-0.872777,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
# Separamos nuestra data
X=data.drop(['churn'], axis=1)
y=data['churn']
X.shape, y.shape

((7043, 15), (7043,))

In [16]:
#Apply SMOTE for upsampling the data
upsampling = SMOTE()
X_smote,y_smote = upsampling.fit_resample(X,y)
y_smote.value_counts()

0.0    5174
1.0    5174
Name: churn, dtype: int64

In [22]:
#Use logistic regression to fit the model and compute the accuracy of the model.
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.30, random_state=40)
model1 = LogisticRegression(random_state=40)
model1.fit(X_train, y_train)
scores=cross_val_score(model1, X_train,y_train,cv=10, n_jobs=-1)
y_pred1=model1.predict(X_test)
print("Train Score:", np.mean(scores))
print("Accuracy test:",metrics.accuracy_score(y_test, y_pred1)) # # Model Accuracy, how often is the classifier correct?

Train Score: 0.7623935987807202
Accuracy test: 0.7739130434782608


In [25]:
#Use decision tree classifier to fit the model and compute the accuracy of the model
X_train, X_test, y_train, y_test = train_test_split(X_smote, y_smote, test_size=0.30, random_state=40)
model2 = DecisionTreeClassifier(random_state=40)
model2.fit(X_train, y_train)
scores2=cross_val_score(model2,X_train, y_train, cv=10,n_jobs=-1)
y_pred2=model2.predict(X_test)
print("Train Score:", np.mean(scores2))
print("Accuracy test:",metrics.accuracy_score(y_test, y_pred2))

Train Score: 0.7731588874071251
Accuracy test: 0.7665056360708534


In [39]:
#Compare the accuracies of the two models
modelos = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree Classifier'],
    "Train Score": [np.mean(scores), np.mean(scores2)],
    "Accuracy test": [metrics.accuracy_score(y_test, y_pred1), metrics.accuracy_score(y_test, y_pred2)]
      }) 

modelos

,Model,Train Score,Accuracy test
0,Logistic Regression,0.762394,0.773913
1,Decision Tree Classifier,0.773159,0.766506


In [27]:
#Apply TomekLinks for downsampling
y.value_counts()

0.0    5174
1.0    1869
Name: churn, dtype: int64

In [32]:
undersample = TomekLinks(sampling_strategy='majority', n_jobs=-1)
X_under,y_under = undersample.fit_resample(X,y)
y_under.value_counts()
# Se reduce 552 muestras

0.0    4622
1.0    1869
Name: churn, dtype: int64

In [33]:
#Use logistic regression to fit the model and compute the accuracy of the model

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_under, y_under, test_size=0.20, random_state=50)
model3 = LogisticRegression(random_state=50)
model3.fit(X_train2, y_train2)
scores3=cross_val_score(model3, X_train2,y_train2,cv=10, n_jobs=-1)
y_pred3=model3.predict(X_test2)
print("Train Score:", np.mean(scores3))
print("Accuracy test:",metrics.accuracy_score(y_test2, y_pred3)) # # Model Accuracy, how often is the classifier correct?

Train Score: 0.8179879946642952
Accuracy test: 0.7929176289453426


In [34]:
#Use decision tree classifier to fit the model and compute the accuracy of the model
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_under, y_under, test_size=0.20, random_state=50)
model4 = DecisionTreeClassifier(random_state=50)
model4.fit(X_train2, y_train2)
scores4=cross_val_score(model4,X_train2, y_train2, cv=10,n_jobs=-1)
y_pred4=model4.predict(X_test2)
print("Train Score:", np.mean(scores4))
print("Accuracy test:",metrics.accuracy_score(y_test2, y_pred4))

Train Score: 0.7598195494293761
Accuracy test: 0.7413394919168591


In [35]:
#Compare the accuracies of the two models
modelos2 = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree Classifier'],
    "Train Score": [np.mean(scores3), np.mean(scores4)],
    "Accuracy test": [metrics.accuracy_score(y_test2, y_pred3), metrics.accuracy_score(y_test2, y_pred4)]
      }) 

modelos2

# DEn este caso , es mejor el score de Logistic Regression que el de Decision Tree

,Model,Train Score,Accuracy test
0,Logistic Regression,0.817988,0.792918
1,Decision Tree Classifier,0.759820,0.741339


In [36]:
#comparamos lso cuatro modelos

modelos3 = pd.DataFrame({
    'Model': ['Logistic Regression with SMOTE','Logistic Regression with Tomeklinks', 'Decision Tree Classifier With SMOTE', 'Decision Tree Classifier With Tomeklinks'],
    "Train Score": [np.mean(scores), np.mean(scores3), np.mean(scores2), np.mean(scores4)],
    "Accuracy test": [metrics.accuracy_score(y_test,y_pred1),metrics.accuracy_score(y_test2, y_pred3),metrics.accuracy_score(y_test, y_pred2),metrics.accuracy_score(y_test2, y_pred4)] }) 

modelos3
#El mejor de los 4 es la regresion logistica con Tomelinks(downsampling)

,Model,Train Score,Accuracy test
0,Logistic Regression with SMOTE,0.762394,0.773913
1,Logistic Regression with Tomeklinks,0.817988,0.792918
2,Decision Tree Classifier With SMOTE,0.773159,0.766506
3,Decision Tree Classifier With Tomeklinks,0.759820,0.741339


In [37]:
#You can also apply this algorithm one more time and check the how the imbalance in the two classes changed from the last time.
y_under.value_counts()

0.0    4622
1.0    1869
Name: churn, dtype: int64

In [38]:
undersample2 = TomekLinks()
X_under2,y_under2 = undersample2.fit_resample(X_under,y_under)
y_under2.value_counts()
# Se reduce (153 muestras) una proporcion menor a la primera vez que lo aplicamos (552)

0.0    4469
1.0    1869
Name: churn, dtype: int64